In [ ]:
import os

import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pyproj as proj
import pandas as pd
from netCDF4 import Dataset  # pylint:disable=no-name-in-module
from shapely.geometry import Point

sf_type = 1
marker_size = 1

# Antarctica

In [ ]:
nc = Dataset(
    os.path.join(
        os.environ["HOME"],
        "Documents",
        "eocis",
        "landice_dash_test",
        "data_files",
        "EOCIS-GIS-L3C-SEC-MULTIMISSION-5KM-5YEAR-MEANS-201001-201501-fv1.nc",
    )
)

In [ ]:
for k, v in nc.variables.items():
    print(k, "\n", v)
    print()

In [ ]:
nc["basin_id"][:].data.shape

In [ ]:
x_values: np.ndarray = nc["x"][:].data
y_values: np.ndarray = nc["y"][:].data
lat: np.ndarray = nc["lat"][:].data
lon: np.ndarray = nc["lon"][:].data
surf_type: np.ndarray = nc["surface_type"][:].data
sec: np.ndarray = np.squeeze(nc["sec"][:].data)

x_coords, y_coords = np.meshgrid(x_values, y_values, indexing="xy")

print(x_values.shape, y_values.shape)
print(x_coords.shape, y_coords.shape, sec.shape, surf_type.shape)

In [ ]:
coords_arr = [Point(x, y) for x, y in zip(x_coords.flatten(), y_coords.flatten())]
sec_arr = sec.flatten()
surf_type_arr = surf_type.flatten()

print(len(coords_arr), sec_arr.shape, surf_type_arr.shape)

In [ ]:
np.unique(surf_type)

In [ ]:
my_data = gpd.GeoDataFrame(
    data={
        "SEC": sec_arr,
        "surface type": surf_type_arr,
        "geometry": coords_arr,
    },
    crs="epsg:3031",
)
my_data.head()

In [ ]:
sum(my_data["SEC"].isna())

In [ ]:
ais_basins = gpd.read_file("aux_files/IMBIE_AIS_Basins/ANT_Basins_IMBIE2_v1.6.shp")
ais_basins = (
    ais_basins.reset_index().rename(columns={"index": "basin_id"}).to_crs("epsg:4326")
)
ais_basins["basin_id"] = (ais_basins["basin_id"] + 1).astype(str)
ais_basins.head()

In [ ]:
# crs_3031 = ccrs.Stereographic(central_latitude=-90, true_scale_latitude=-71)
# transformed = my_data.to_crs(crs_3031)

# fig, ax = plt.subplots(
#     figsize=(8, 6), facecolor="white", subplot_kw=dict(projection=crs_3031)
# )  # Create our plot

# transformed[transformed["SEC"].notna()].plot(
#     column="SEC",
#     ax=ax,
#     legend=True,
#     vmax=2,
#     vmin=-2,
#     marker="s",
#     markersize=marker_size,
#     cmap="bwr_r",
# )

# ais_basins.plot(color="none", edgecolor="black", ax=ax, alpha=0.5, lw=0.7)

# # ax.coastlines(resolution="50m", color="black")  # Add coastlines
# gl = ax.gridlines(draw_labels=True, color="black", alpha=0.25)
# gl.ylabel_style = {
#     "color": "black",
#     "alpha": 0.5,
# }
# ax.set_xlim(-2.8e6, 2.8e6)
# ax.set_ylim(-2.8e6, 2.8e6)

# fig.show()

In [ ]:
timedata_ais = pd.read_csv(
    "/home/jgnq4/Documents/eocis/landice_dash_test/processed_files/time_series_data_AIS.csv"
).sort_values("code")

max_v_ais = np.round(timedata_ais["SEC"].abs().max(axis=None) + 0.05, 1)
timedata_ais = timedata_ais.reset_index(drop=True).reset_index()
timedata_ais["basin"] = timedata_ais["basin"].astype(str)
timedata_ais = pd.merge(
    timedata_ais,
    ais_basins[["basin_id", "Subregion"]],
    how="outer",
    left_on="basin",
    right_on="basin_id",
).drop(columns="basin_id")
timedata_ais.loc[timedata_ais["basin"] == "all", "Subregion"] = "All"
timedata_ais.loc[timedata_ais["basin"] == "0", "Subregion"] = "Outside Basins"
timedata_ais

In [ ]:
import altair as alt
import vl_convert as vlc
import json
from IPython.display import Image

alt.data_transformers.enable("vegafusion")

click_basin_ais = alt.selection_point(fields=["Subregion"])

map_ais = (
    alt.Chart(
        ais_basins,
    )
    .mark_geoshape()
    .encode(
        color=alt.Color("Subregion:N").legend(orient="right"),
        opacity=alt.condition(click_basin_ais, alt.value(1), alt.value(0.2)),
        tooltip=["basin_id:N", "Subregion:N", "Regions:N"],
    )
    .project(type="stereographic", center=[-90, -180])
    .properties(width=300, height=300)
)

jchart1 = alt.JupyterChart(map_ais.add_params(click_basin_ais))
jchart1

In [ ]:
# nearest = alt.selection_point(nearest=True, on="mouseover", fields=["index"], empty=False)

line_ais = (
    alt.Chart(timedata_ais, title="Mean SEC per Basin")
    .mark_line()
    .encode(
        alt.X("midpoint:Q", axis=alt.Axis(labels=False), title="Time Period"),
        alt.Y(
            "SEC:Q",
            scale=alt.Scale(domain=(-max_v_ais, max_v_ais)),
            title="Mean elevation change (m/year)",
        ),
        opacity=alt.condition(click_basin_ais, alt.value(1), alt.value(0.05)),
        color=alt.Color("Subregion:N"),
        tooltip=["SEC", "period", "Subregion"],
    )
    .properties(width=900, height=300)
)

# # Draw a rule at the location of the selection
# rules = base.mark_rule(color="gray").encode(
#     x="index:Q",
#     opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
#     tooltip=[
#         alt.Tooltip("period", type="nominal", title="Time period"),
#     ],
# )

# points = line.mark_circle().encode(opacity=alt.condition(nearest, alt.value(1), alt.value(0)))

zero = pd.DataFrame([{"zero": 0.0}])
zero_rule = (
    alt.Chart(zero).mark_rule(color="black", strokeDash=[1]).encode(y="zero:Q", size=alt.value(1))
)

line_chart_ais = line_ais + zero_rule

jchart2 = alt.JupyterChart(line_chart_ais.add_params(click_basin_ais))
jchart2

In [ ]:
chart = (line_chart_ais | map_ais).add_params(click_basin_ais)


jchart = alt.JupyterChart(chart)
jchart

# Greenland

In [ ]:
nc = Dataset(
    os.path.join(
        os.environ["HOME"],
        "Documents",
        "eocis",
        "landice_dash_test",
        "data_files",
        "EOCIS-GIS-L3C-SEC-MULTIMISSION-5KM-5YEAR-MEANS-201001-201501-fv1.nc",
    )
)

In [ ]:
for k, v in nc.variables.items():
    print(k, "\n", v)
    print()

In [ ]:
x_values: np.ndarray = nc["x"][:].data
y_values: np.ndarray = nc["y"][:].data
lat: np.ndarray = nc["lat"][:].data
lon: np.ndarray = nc["lon"][:].data
surf_type: np.ndarray = nc["surface_type"][:].data
sec: np.ndarray = np.squeeze(nc["sec"][:].data)

x_coords, y_coords = np.meshgrid(x_values, y_values, indexing="xy")

print(x_values.shape, y_values.shape)
print(x_coords.shape, y_coords.shape, sec.shape, surf_type.shape)
print("X:", np.min(x_values), np.max(x_values))
print("Y:", np.min(y_values), np.max(y_values))

In [ ]:
coords_arr = [Point(x, y) for x, y in zip(x_coords.flatten(), y_coords.flatten())]
sec_arr = sec.flatten()
surf_type_arr = surf_type.flatten()

print(len(coords_arr), sec_arr.shape, surf_type_arr.shape)

In [ ]:
my_data = gpd.GeoDataFrame(
    data={
        "SEC": sec_arr,
        "surface type": surf_type_arr,
        "geometry": coords_arr,
    },
    crs="epsg:3413",
)
my_data.head()

In [ ]:
crs_new = ccrs.NorthPolarStereo(central_longitude=-40)
gis_basins = gpd.read_file("aux_files/IMBIE_GIS_Basins/Greenland_Basins_PS_v1.4.2.shp").to_crs(
    "EPSG:4326"
)
gis_basins = gis_basins.reset_index().rename(
    columns={"index": "basin_id", "SUBREGION1": "Subregion"}
)
gis_basins["basin_id"] = (gis_basins["basin_id"] + 1).astype(str)
gis_basins.head(10)

In [ ]:
crs_new = ccrs.NorthPolarStereo(central_longitude=-40)
transformed = my_data.to_crs(crs_new)

fig, ax = plt.subplots(
    figsize=(9, 7), facecolor="white", subplot_kw=dict(projection=crs_new)
)  # Create our plot

transformed[transformed["SEC"].notna() & (transformed["surface type"] == 0)].plot(
    column="SEC",
    ax=ax,
    legend=True,
    vmax=2,
    vmin=-2,
    marker="s",
    markersize=marker_size,
    cmap="bwr_r",
)

gis_basins.to_crs(crs_new).plot(color="none", edgecolor="black", ax=ax, alpha=0.5, lw=0.7)

gl = ax.gridlines(draw_labels=True, color="black", alpha=0.25)
gl.ylabel_style = {
    "color": "black",
    "alpha": 0.5,
}


fig.show()

In [ ]:
click_basin_gis = alt.selection_point(fields=["Subregion"], bind="legend")

map_gis = (
    alt.Chart(
        gis_basins.to_crs(epsg="4326"),
    )
    .mark_geoshape()
    .encode(
        color=alt.Color(
            "Subregion:N",
            sort=["All", "Outside basin", "NO", "NE", "NW", "CE", "CW", "SE", "SW"],
        ).legend(orient="right"),
        opacity=alt.condition(click_basin_gis, alt.value(1), alt.value(0.2)),
        tooltip=["basin_id:N", "Subregion:N"],
    )
    .project("stereographic", rotate=[0, 45, 20])
)

jchart4 = alt.JupyterChart(map_gis.add_params(click_basin_gis), debounce_wait=1000)
jchart4

In [ ]:
timedata_gis = pd.read_csv(
    "/home/jgnq4/Documents/eocis/landice_dash_test/processed_files/time_series_data_GIS.csv"
).sort_values("code")

max_v_gis = np.round(timedata_gis["SEC"].abs().max(axis=None) + 0.05, 1)
timedata_gis = timedata_gis.reset_index(drop=True).reset_index()
timedata_gis["basin"] = timedata_gis["basin"].astype(str)
timedata_gis = pd.merge(
    timedata_gis,
    gis_basins[["basin_id", "Subregion"]],
    how="outer",
    left_on="basin",
    right_on="basin_id",
).drop(columns="basin_id")
timedata_gis.loc[timedata_gis["basin"] == "all", "Subregion"] = "All"
timedata_gis = timedata_gis.drop(np.where(timedata_gis["basin"] == "0")[0], axis=0)
# timedata_gis.loc[timedata_gis["basin"] == "0", "Subregion"] = "Outside"

timedata_gis

In [ ]:
max_v_gis

In [ ]:
timedata_gis[timedata_gis["basin"] == "7"]

In [ ]:
line_gis = (
    alt.Chart(timedata_gis, title="Mean SEC per Basin")
    .mark_line()
    .encode(
        alt.X(
            "midpoint:Q",
            axis=alt.Axis(format="4d", tickMinStep=2),
            title="Time Period",
        ),
        alt.Y(
            "SEC:Q",
            scale=alt.Scale(domain=(-max_v_gis, max_v_gis)),
            title="Mean elevation change (m/year)",
        ),
        opacity=alt.condition(click_basin_gis, alt.value(1), alt.value(0.05)),
        color=alt.Color(
            "Subregion:N",
            sort=["All", "Outside basin", "NO", "NE", "NW", "CE", "CW", "SE", "SW"],
        ).legend(orient="right"),
        tooltip=["SEC", "period", "Subregion"],
    )
    .properties(width=900, height=300)
)

# # Draw a rule at the location of the selection
# rules = base.mark_rule(color="gray").encode(
#     x="index:Q",
#     opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
#     tooltip=[
#         alt.Tooltip("period", type="nominal", title="Time period"),
#     ],
# )

# points = line.mark_circle().encode(opacity=alt.condition(nearest, alt.value(1), alt.value(0)))

zero = pd.DataFrame([{"zero": 0.0}])
zero_rule = (
    alt.Chart(zero)
    .mark_rule(color="black", strokeDash=[1])
    .encode(y="zero:Q", size=alt.value(1))
)

line_chart_gis = line_gis + zero_rule

jchart4 = alt.JupyterChart(line_chart_gis.add_params(click_basin_gis))
jchart4

In [ ]:
chart = (line_chart_gis | map_gis).add_params(click_basin_gis)


jchart = alt.JupyterChart(chart)
jchart